In [1]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

In [2]:
#定义生成的锚框
#@save
def multibox_prior(data, sizes, ratios):
    """生成以每个像素为中心具有不同形状的锚框"""
    in_height, in_width = data.shape[-2:]
    device, num_sizes, num_ratios = data.device, len(sizes), len(ratios)
    boxes_per_pixel = (num_sizes + num_ratios - 1)
    size_tensor = torch.tensor(sizes, device=device)
    ratio_tensor = torch.tensor(ratios, device=device)

    # 为了将锚点移动到像素的中心，需要设置偏移量。
    # 因为一个像素的高为1且宽为1，我们选择偏移我们的中心0.5
    offset_h, offset_w = 0.5, 0.5
    steps_h = 1.0 / in_height  # 在y轴上缩放步长
    steps_w = 1.0 / in_width  # 在x轴上缩放步长

    # 生成锚框的所有中心点
    center_h = (torch.arange(in_height, device=device) + offset_h) * steps_h
    center_w = (torch.arange(in_width, device=device) + offset_w) * steps_w
    shift_y, shift_x = torch.meshgrid(center_h, center_w, indexing='ij')
    shift_y, shift_x = shift_y.reshape(-1), shift_x.reshape(-1)

    # 生成“boxes_per_pixel”个高和宽，
    # 之后用于创建锚框的四角坐标(xmin,xmax,ymin,ymax)
    w = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]),
                   sizes[0] * torch.sqrt(ratio_tensor[1:])))\
                   * in_height / in_width  # 处理矩形输入
    h = torch.cat((size_tensor / torch.sqrt(ratio_tensor[0]),
                   sizes[0] / torch.sqrt(ratio_tensor[1:])))
    # 除以2来获得半高和半宽
    anchor_manipulations = torch.stack((-w, -h, w, h)).T.repeat(
                                        in_height * in_width, 1) / 2

    # 每个中心点都将有“boxes_per_pixel”个锚框，
    # 所以生成含所有锚框中心的网格，重复了“boxes_per_pixel”次
    out_grid = torch.stack([shift_x, shift_y, shift_x, shift_y],
                dim=1).repeat_interleave(boxes_per_pixel, dim=0)
    output = out_grid + anchor_manipulations
    return output.unsqueeze(0)

In [3]:
#如何显示锚框
#@save
def show_bboxes(axes, bboxes, labels=None, colors=None):
    """显示所有边界框"""
    def _make_list(obj, default_values=None):
        if obj is None:
            obj = default_values
        elif not isinstance(obj, (list, tuple)):
            obj = [obj]
        return obj

    labels = _make_list(labels)
    colors = _make_list(colors, ['b', 'g', 'r', 'm', 'c'])
    for i, bbox in enumerate(bboxes):
        color = colors[i % len(colors)]
        rect = d2l.bbox_to_rect(bbox.detach().numpy(), color)
        axes.add_patch(rect)
        if labels and len(labels) > i:
            text_color = 'k' if color == 'w' else 'w'
            axes.text(rect.xy[0], rect.xy[1], labels[i],
                      va='center', ha='center', fontsize=9, color=text_color,
                      bbox=dict(facecolor=color, lw=0))

clamp函数
将输入的丈量夹在min,max之间

In [12]:
A=torch.arange(12).reshape(1,3,4)
print(A)
B=A.clamp(4,7)
print(B)

C=torch.rand(12).reshape(6,1,2)
print("C: ",C)

D=torch.rand(10).reshape(1,5,2)
print("D: ",D)
E=torch.max(C,D)
print(E)
print("E shape:" ,E.shape)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]])
tensor([[[4, 4, 4, 4],
         [4, 5, 6, 7],
         [7, 7, 7, 7]]])
C:  tensor([[[0.5922, 0.9701]],

        [[0.5435, 0.2092]],

        [[0.7543, 0.2658]],

        [[0.6739, 0.1531]],

        [[0.7501, 0.8845]],

        [[0.4140, 0.2682]]])
D:  tensor([[[0.5348, 0.4766],
         [0.6504, 0.4377],
         [0.7617, 0.4411],
         [0.1521, 0.4689],
         [0.0174, 0.3193]]])
tensor([[[0.5922, 0.9701],
         [0.6504, 0.9701],
         [0.7617, 0.9701],
         [0.5922, 0.9701],
         [0.5922, 0.9701]],

        [[0.5435, 0.4766],
         [0.6504, 0.4377],
         [0.7617, 0.4411],
         [0.5435, 0.4689],
         [0.5435, 0.3193]],

        [[0.7543, 0.4766],
         [0.7543, 0.4377],
         [0.7617, 0.4411],
         [0.7543, 0.4689],
         [0.7543, 0.3193]],

        [[0.6739, 0.4766],
         [0.6739, 0.4377],
         [0.7617, 0.4411],
         [0.6739, 0.4689],
        

In [17]:
A=torch.arange(6)

A=A[:,None]
print(A)
print(A.shape)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]])
torch.Size([6, 1])


In [18]:
#Iou计算
#@save

#首先这个输入定义为2维度
def box_iou(boxes1, boxes2):
    """计算两个锚框或边界框列表中成对的交并比"""
    box_area = lambda boxes: ((boxes[:, 2] - boxes[:, 0]) *
                              (boxes[:, 3] - boxes[:, 1]))
    # boxes1,boxes2,areas1,areas2的形状:
    # boxes1：(boxes1的数量,4),   xmin,ymin,xmax,ymax
    # boxes2：(boxes2的数量,4),
    # areas1：(boxes1的数量,),
    # areas2：(boxes2的数量,)   
    areas1 = box_area(boxes1)
    areas2 = box_area(boxes2)
    # inter_upperlefts,inter_lowerrights,inters的形状:
    # (boxes1的数量,boxes2的数量,2)
    inter_upperlefts = torch.max(boxes1[:, None, :2], boxes2[:, :2])
#boxes1[:, None, :2]  为[anchor_num,1,2]
#boxes2[class_num,2] 也可以是[1,class_num,2]
#inter_upperlefts [anchor_num,class_num,2]

    inter_lowerrights = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])

#inter_lowerrights [anchor_num,class_num,2]
    inters = (inter_lowerrights - inter_upperlefts).clamp(min=0)
    # inter_areasandunion_areas的形状:(boxes1的数量,boxes2的数量)
    inter_areas = inters[:, :, 0] * inters[:, :, 1]
    #这个地方选出了0，inters[:,:,0].shape 为【anchor_num,class_num】
    union_areas = areas1[:, None] + areas2 - inter_areas

    #这里再次用到了广播机制
    return inter_areas / union_areas

#返回【anchorNum,class_num

将正式框赋值给锚框

In [35]:
A=torch.full((10,),-1,dtype=torch.long)
print(A)
print(A.shape)
print(A.dim())
B=torch.rand(10)
print(B.dim())
C=torch.rand(10).reshape(10,1)
print(C)


tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
torch.Size([10])
1
1
tensor([[0.8248],
        [0.8070],
        [0.5603],
        [0.3608],
        [0.5355],
        [0.9579],
        [0.6616],
        [0.7839],
        [0.8681],
        [0.5668]])


In [38]:
A=torch.arange(12).reshape(3,4)
print(A)    
print(A.max(dim=0))
print(A.sum(dim=0))

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
torch.return_types.max(
values=tensor([ 8,  9, 10, 11]),
indices=tensor([2, 2, 2, 2]))
tensor([12, 15, 18, 21])


In [44]:
A=torch.tensor([1,2,4,2,5,8,2])

print(A<=3)

torch.nonzero(A <=3).reshape(-1)

tensor([ True,  True, False,  True, False, False,  True])


tensor([0, 1, 3, 6])

In [51]:
A=torch.rand(12).reshape(3,4)
print(A)
print(torch.argmax(A))

tensor([[0.5088, 0.2500, 0.7676, 0.1080],
        [0.2566, 0.9097, 0.4777, 0.8445],
        [0.9234, 0.0722, 0.3108, 0.4472]])
tensor(8)


In [58]:
A=torch.zeros(12,dtype=torch.long)
print(
    A
)
B=torch.tensor([0,3],dtype=torch.long)
C=torch.tensor([100,200],dtype=torch.long)
A[B]=C
print(A)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([100,   0,   0, 200,   0,   0,   0,   0,   0,   0,   0,   0])


In [59]:
#@save
def assign_anchor_to_bbox(ground_truth, anchors, device, iou_threshold=0.5):
    #ground_truth 真实框
    #anchors 所有的锚框
    """将最接近的真实边界框分配给锚框"""
    num_anchors, num_gt_boxes = anchors.shape[0], ground_truth.shape[0]
    #num_anchors 为锚框的个数，num_gt_boxes为真实框的个数
    # 位于第i行和第j列的元素x_ij是锚框i和真实边界框j的IoU
    jaccard = box_iou(anchors, ground_truth)
    #返回的是（anchors_num,ground_truth_num) 的Shape；

    # 对于每个锚框，分配的真实边界框的张量
    anchors_bbox_map = torch.full((num_anchors,), -1, dtype=torch.long,
                                  device=device)
    #这个地方的anchors_bbox_map是一个一维度tensor; 个数为所有anchor的个数


    # 根据阈值，决定是否分配真实边界框
    max_ious, indices = torch.max(jaccard, dim=1)
    #这里横向的做最大值计算，并返回最大值与Index；  单个锚框与各个ground_truth的最大值；


    anc_i = torch.nonzero(max_ious >= iou_threshold).reshape(-1)

    #这里返回max_ious中满足max_ious>=iou_threshold情况下的索引列表

    box_j = indices[max_ious >= iou_threshold]

    #这里返回类别的索引；


    anchors_bbox_map[anc_i] = box_j   #这个地方实际把每个anchor 对应的最大类赋值； 通过后续的更正，有可能需要更改，box_j有可能会变成一行中第二大的那个数；

    #保存每个anchor对应的最大类别


    col_discard = torch.full((num_anchors,), -1)
    row_discard = torch.full((num_gt_boxes,), -1)

    for _ in range(num_gt_boxes):
        max_idx = torch.argmax(jaccard)
        box_idx = (max_idx % num_gt_boxes).long()
        anc_idx = (max_idx / num_gt_boxes).long()
        anchors_bbox_map[anc_idx] = box_idx
        jaccard[:, box_idx] = col_discard
        jaccard[anc_idx, :] = row_discard
    return anchors_bbox_map

#返回一个一维tensor ；每个真是框赋值给

In [66]:
A=torch.arange(12).reshape(3,4)
B=torch.rand(12).reshape(3,4)
print(torch.cat([A,B],dim=1))
print(torch.cat([A,B],axis=1))

tensor([[ 0.0000,  1.0000,  2.0000,  3.0000,  0.7552,  0.8066,  0.9338,  0.1206],
        [ 4.0000,  5.0000,  6.0000,  7.0000,  0.9740,  0.1821,  0.6149,  0.8341],
        [ 8.0000,  9.0000, 10.0000, 11.0000,  0.9786,  0.3828,  0.4389,  0.7362]])
tensor([[ 0.0000,  1.0000,  2.0000,  3.0000,  0.7552,  0.8066,  0.9338,  0.1206],
        [ 4.0000,  5.0000,  6.0000,  7.0000,  0.9740,  0.1821,  0.6149,  0.8341],
        [ 8.0000,  9.0000, 10.0000, 11.0000,  0.9786,  0.3828,  0.4389,  0.7362]])


In [60]:
#下面的例子，是anchor框，与真实的框偏移量的计算；
#@save
def offset_boxes(anchors, assigned_bb, eps=1e-6):
    """对锚框偏移量的转换"""
    c_anc = d2l.box_corner_to_center(anchors)
    c_assigned_bb = d2l.box_corner_to_center(assigned_bb)

    offset_xy = 10 * (c_assigned_bb[:, :2] - c_anc[:, :2]) / c_anc[:, 2:]
    offset_wh = 5 * torch.log(eps + c_assigned_bb[:, 2:] / c_anc[:, 2:])

    offset = torch.cat([offset_xy, offset_wh], axis=1)

    #最后返回Shape 为(num_anchor,4)

    return offset

In [ ]:
#@save
def multibox_target(anchors, labels):
    """使用真实边界框标记锚框"""
    batch_size, anchors = labels.shape[0], anchors.squeeze(0)
    batch_offset, batch_mask, batch_class_labels = [], [], []

    device, num_anchors = anchors.device, anchors.shape[0]
    
    for i in range(batch_size):
        label = labels[i, :, :]
        anchors_bbox_map = assign_anchor_to_bbox(
            label[:, 1:], anchors, device)
        bbox_mask = ((anchors_bbox_map >= 0).float().unsqueeze(-1)).repeat(
            1, 4)
        # 将类标签和分配的边界框坐标初始化为零
        class_labels = torch.zeros(num_anchors, dtype=torch.long,
                                   device=device)
        assigned_bb = torch.zeros((num_anchors, 4), dtype=torch.float32,
                                  device=device)
        # 使用真实边界框来标记锚框的类别。
        # 如果一个锚框没有被分配，标记其为背景（值为零）
        indices_true = torch.nonzero(anchors_bbox_map >= 0)
        bb_idx = anchors_bbox_map[indices_true]
        class_labels[indices_true] = label[bb_idx, 0].long() + 1
        assigned_bb[indices_true] = label[bb_idx, 1:]
        # 偏移量转换
        offset = offset_boxes(anchors, assigned_bb) * bbox_mask
        batch_offset.append(offset.reshape(-1))
        batch_mask.append(bbox_mask.reshape(-1))
        batch_class_labels.append(class_labels)
    bbox_offset = torch.stack(batch_offset)
    bbox_mask = torch.stack(batch_mask)
    class_labels = torch.stack(batch_class_labels)
    return (bbox_offset, bbox_mask, class_labels)